<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MY-AI-GIZMO INTERACTIVE LAUNCHER WITH PERSISTENT STORAGE
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
from pathlib import Path
import shutil

print("🚀 MY-AI-GIZMO INTERACTIVE INSTALLER")
print("="*70)

# --- STEP 1: Mount Google Drive ---
print("\n📂 Step 1: Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("   ✅ Google Drive mounted")

# --- STEP 2: Setup Persistent Directories ---
print("\n📁 Step 2: Setting up persistent storage...")

DRIVE_BASE = Path("/content/drive/MyDrive/MY-AI-Gizmo")
DRIVE_BASE.mkdir(exist_ok=True)

REPO_DIR = DRIVE_BASE / "MY-AI-Gizmo-working"
MODELS_DIR = DRIVE_BASE / "models"
USER_DATA_DIR = DRIVE_BASE / "user_data"

MODELS_DIR.mkdir(exist_ok=True)
USER_DATA_DIR.mkdir(exist_ok=True)

print(f"   📂 Installation: {DRIVE_BASE}")

already_installed = REPO_DIR.exists() and (REPO_DIR / "server.py").exists()
model_exists = (MODELS_DIR / "llama-2-7b.Q4_K_M.gguf").exists()

if already_installed:
    print("   ✅ Installation found")
if model_exists:
    print("   ✅ Model found")

# --- STEP 3: Clone Repository ---
if not already_installed:
    print("\n📥 Step 3: Cloning repository...")
    if REPO_DIR.exists():
        shutil.rmtree(REPO_DIR)

    os.chdir(DRIVE_BASE)
    subprocess.run([
        "git", "clone",
        "https://github.com/gitleon8301/MY-AI-Gizmo-working.git"
    ], check=True)
    print("   ✅ Repository cloned")
else:
    print("\n✅ Step 3: Using existing installation")

os.chdir(REPO_DIR)

# --- STEP 4: Link User Data ---
print("\n🔗 Step 4: Setting up chat persistence...")

local_user_data = REPO_DIR / "user_data"

if local_user_data.exists() and not local_user_data.is_symlink():
    print("   📦 Backing up user_data...")
    for item in local_user_data.rglob("*"):
        if item.is_file():
            rel_path = item.relative_to(local_user_data)
            dest = USER_DATA_DIR / rel_path
            dest.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(item, dest)
    shutil.rmtree(local_user_data)

if not local_user_data.exists():
    local_user_data.symlink_to(USER_DATA_DIR)
    print("   ✅ Linked to Google Drive")

(USER_DATA_DIR / "logs" / "chat").mkdir(parents=True, exist_ok=True)
(USER_DATA_DIR / "logs" / "instruct").mkdir(parents=True, exist_ok=True)

# --- STEP 5: Environment ---
print("\n🔧 Step 5: Configuring environment...")
os.environ['MPLBACKEND'] = 'Agg'
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
print("   ✅ Environment configured")

# --- STEP 6: Install Dependencies ---
print("\n📦 Step 6: Installing dependencies...")

subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "-q"],
               check=False, stdout=subprocess.DEVNULL)

req_files = [
    "requirements_cpu_only.txt",
    "requirements.txt",
]

installed = False
for req_file in req_files:
    if Path(req_file).exists():
        print(f"   Installing from {req_file}...")
        try:
            result = subprocess.run([
                sys.executable, "-m", "pip", "install",
                "-r", req_file, "-q"
            ], check=True, timeout=300, capture_output=True, text=True)
            installed = True
            print(f"   ✅ Installed from {req_file}")
            break
        except subprocess.CalledProcessError as e:
            print(f"   ⚠️  Error: {e.stderr[:200]}")
            continue

if not installed:
    print("   Installing core packages...")
    core = [
        "torch", "transformers", "gradio",
        "accelerate", "markdown", "numpy", "Pillow",
        "pyyaml", "requests", "safetensors",
        "sentencepiece", "tqdm", "protobuf", "llama-cpp-python"
    ]
    for pkg in core:
        subprocess.run([sys.executable, "-m", "pip", "install", pkg, "-q"],
                      check=False, stdout=subprocess.DEVNULL)

print("   ✅ Dependencies installed")

# --- STEP 7: Fix UI ---
print("\n🔧 Step 7: Patching UI...")

ui_file = Path("modules/ui.py")
if ui_file.exists():
    with open(ui_file, 'r', encoding='utf-8') as f:
        content = f.read()

    if 'button_shadow_hover' in content and 'if False:  # Disabled' not in content:
        content = content.replace(
            'if not shared.args.old_colors:',
            'if False:  # Disabled for Colab'
        )
        with open(ui_file, 'w', encoding='utf-8') as f:
            f.write(content)
        print("   ✅ UI patched")
    else:
        print("   ✅ UI compatible")

# --- STEP 8: Download Model ---
print("\n⬇️  Step 8: Checking model...")

MODEL_FILE = MODELS_DIR / "llama-2-7b.Q4_K_M.gguf"

if MODEL_FILE.exists():
    print(f"   ✅ Model exists ({MODEL_FILE.stat().st_size / (1024**3):.2f} GB)")
else:
    print("   📥 Downloading model...")
    model_url = "https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf"
    subprocess.run([
        "wget", "-q", "--show-progress",
        "-O", str(MODEL_FILE),
        model_url
    ], check=True)
    print("   ✅ Model downloaded")

# --- STEP 9: Link Models ---
repo_models = REPO_DIR / "models"
if not repo_models.is_symlink():
    if repo_models.exists():
        shutil.rmtree(repo_models)
    repo_models.symlink_to(MODELS_DIR)

# --- STEP 10: Interactive Configuration ---
print("\n" + "="*70)
print("⚙️  CONFIGURATION")
print("="*70)

# Get hardware mode
print("\n🖥️  Select Hardware Mode:")
print("1. CPU (Slower, works on any Colab)")
print("2. GPU (Faster, requires GPU runtime)")
print("3. Auto-detect")

while True:
    hardware_choice = input("\nEnter choice (1-3): ").strip()
    if hardware_choice in ['1', '2', '3']:
        break
    print("❌ Invalid choice. Please enter 1, 2, or 3.")

if hardware_choice == '1':
    use_cpu = True
    print("✅ Using CPU mode")
elif hardware_choice == '2':
    use_cpu = False
    print("✅ Using GPU mode")
else:
    # Auto-detect
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True)
        use_cpu = result.returncode != 0
        if use_cpu:
            print("ℹ️  No GPU detected - using CPU")
        else:
            print("✅ GPU detected - using GPU")
    except:
        use_cpu = True
        print("ℹ️  Auto-detection failed - using CPU")

# Get interface mode
print("\n💬 Select Interface Mode:")
print("1. Chat (Conversational AI)")
print("2. Instruct (Command-based)")
print("3. Default (Chat)")

while True:
    mode_choice = input("\nEnter choice (1-3): ").strip()
    if mode_choice in ['1', '2', '3']:
        break
    print("❌ Invalid choice. Please enter 1, 2, or 3.")

if mode_choice == '2':
    interface_mode = '--instruct'
    print("✅ Using Instruct mode")
else:
    interface_mode = '--chat'
    print("✅ Using Chat mode")

# Additional options
print("\n🔧 Additional Options:")
print("1. Enable API (allows external access)")
print("2. Verbose logging (more detailed output)")
print("3. Both")
print("4. None")

while True:
    options_choice = input("\nEnter choice (1-4): ").strip()
    if options_choice in ['1', '2', '3', '4']:
        break
    print("❌ Invalid choice. Please enter 1, 2, 3, or 4.")

extra_args = []
if options_choice in ['1', '3']:
    extra_args.extend(['--api', '--public-api'])
    print("✅ API enabled")
if options_choice in ['2', '3']:
    extra_args.append('--verbose')
    print("✅ Verbose logging enabled")

# --- STEP 11: Build Launch Command ---
print("\n" + "="*70)
print("🚀 LAUNCHING MY-AI-GIZMO")
print("="*70)

# Build command
launch_cmd = [sys.executable, "server.py"]

# Add hardware mode
if use_cpu:
    launch_cmd.append("--cpu")
else:
    launch_cmd.append("--auto-devices")

# Add interface mode
launch_cmd.append(interface_mode)

# Add standard args
launch_cmd.extend([
    "--listen",
    "--share",
    "--model", str(MODEL_FILE)
])

# Add extra args
launch_cmd.extend(extra_args)

print(f"\n📝 Launch Configuration:")
print(f"   Hardware: {'CPU' if use_cpu else 'GPU'}")
print(f"   Interface: {'Chat' if '--chat' in launch_cmd else 'Instruct'}")
print(f"   API: {'Enabled' if '--api' in launch_cmd else 'Disabled'}")
print(f"   Verbose: {'Yes' if '--verbose' in launch_cmd else 'No'}")
print(f"   Model: {MODEL_FILE.name}")

print(f"\n💾 Chats save to: {USER_DATA_DIR / 'logs'}")
print("\n" + "="*70)
print("⏳ Starting server (may take 1-2 minutes)...")
print("🔗 LOOK FOR THE GRADIO LINK BELOW:")
print("="*70 + "\n")

# --- STEP 12: Launch with Error Handling ---
try:
    # First, check if server.py exists and is valid
    if not Path("server.py").exists():
        print("❌ ERROR: server.py not found!")
        print(f"Current directory: {os.getcwd()}")
        print("\nDirectory contents:")
        subprocess.run(["ls", "-la"], check=False)
        sys.exit(1)

    # Try to run with captured output for better error messages
    print("Starting server...\n")
    process = subprocess.Popen(
        launch_cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )

    # Print output in real-time
    for line in process.stdout:
        print(line, end='')
        # Check for Gradio link
        if 'gradio.live' in line.lower() or 'running on' in line.lower():
            print("\n" + "="*70)
            print("✅ SERVER RUNNING! Use the link above ☝️")
            print("="*70)

    process.wait()

except KeyboardInterrupt:
    print("\n\n⏹️  Server stopped by user")
    print(f"💾 Chats saved in: {USER_DATA_DIR / 'logs'}")

except Exception as e:
    print(f"\n\n❌ ERROR: {e}")
    print("\n🔍 Troubleshooting:")
    print(f"   • Working directory: {os.getcwd()}")
    print(f"   • Model exists: {MODEL_FILE.exists()}")
    print(f"   • Python: {sys.executable}")

    # Try to get more error info
    print("\n📋 Trying to run with error details...")
    result = subprocess.run(
        launch_cmd,
        capture_output=True,
        text=True
    )
    print("\nSTDOUT:")
    print(result.stdout)
    print("\nSTDERR:")
    print(result.stderr)

    print("\n💡 Common fixes:")
    print("   1. Restart runtime and try again")
    print("   2. Check if requirements installed correctly")
    print("   3. Try running: !python server.py --help")